In [1]:
%pwd

'd:\\ML_Deployment\\Automadata\\research'

In [2]:
import os 
os.chdir('../')
%pwd


'd:\\ML_Deployment\\Automadata'

In [3]:
from dataclasses import dataclass
from pathlib import Path
from src.logging import logger
import pandas as pd
from src.constants import *
from src.utils.common import read_yaml, create_directories


In [35]:
@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE : str
    data_path : Path
    all_schema : dict

In [36]:
class ConfigManager:
    
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation 
        schema = self.schema.COLUMNS
        
        create_directories([config.root_dir])
        
        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            data_path= config.data_path,
            STATUS_FILE  = config.STATUS_FILE,
            all_schema = schema  
        )
        return data_validation_config

In [37]:
class DataValidation:
    def __init__(self, config:DataValidationConfig):
        self.config = config
    
    def validate_features(self)->bool:
        
        try:
            validation_status = None
            
            data = pd.read_csv(self.config.data_path)
            
            all_cols = list(data.columns)
            
            all_schema = self.config.all_schema.keys()
            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status : {validation_status}")
                
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status : {validation_status}")
            
            return validation_status
        
        except Exception as e:
            raise e

In [38]:
try:
    config = ConfigManager()
    data_valid_config = config.get_data_validation_config()
    data_valid = DataValidation(config=data_valid_config)
    data_valid.validate_features()
except Exception as e:
    raise e

[2024-01-12 10:35:36,444: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-12 10:35:36,444: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-12 10:35:36,456: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-12 10:35:36,457: INFO: common: created directory at: artifacts]
[2024-01-12 10:35:36,458: INFO: common: created directory at: artifacts/data_validation]
